NOTE - NOT YET COMPLETED

# Install Data & Packages

In [45]:
pip install statsmodels --quiet

Import necessary libraries.

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS
from sklearn.preprocessing import StandardScaler
import os
import statsmodels as sm
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from numpy.linalg import inv, eig
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
try:
    import statsmodels.api.add_constant
except ImportError:
    pass

Import Data.

In [47]:
path_data = '/content/data.csv' #edit path as necessary

# Check if files exist before loading
if os.path.exists(path_data):
    data = pd.read_csv(path_data)
    print("Data loaded successfully.")
else:
    print("Error: One or both files not found. Check the file paths.")

Data loaded successfully.


In [48]:
data

,ViolentCrimesPerPop,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,0.20,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,0.67,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,0.43,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,0.12,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,0.03,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.09,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.36
1990,0.45,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.23
1991,0.23,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.22
1992,0.19,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.27


In [49]:
y = data.loc[:,'ViolentCrimesPerPop']
X = data.loc[:, data.columns != 'ViolentCrimesPerPop']

# Least Squares

In [50]:
p = X.shape[1]
n = X.shape[0]
print(f'The number of observations: {n}.\nThe number of predictors: {p}.\nSince n > p, we can expect LS to yield a unique solution.')

The number of observations: 1994.
The number of predictors: 100.
Since n > p, we can expect LS to yield a unique solution.


In [51]:
if 'constants' not in X.columns:
  const = pd.DataFrame([1]*X.shape[0]).rename(columns = {0: 'constants'})
  X = pd.concat([const, X], axis = 1)
X

,constants,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,1,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.36
1990,1,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.23
1991,1,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.22
1992,1,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.27


In [52]:
#Least squares using statsmodels
lsmodel = OLS(y,X).fit()
summary = lsmodel.summary()

#Access the coefficients and p-values
coefficients = lsmodel.params
p_values = lsmodel.pvalues

In [53]:
alpha_values = [0.10, 0.05, 0.01, 0.001]
filtered_results_dict = {alpha:[] for alpha in alpha_values}
for alpha in alpha_values:
  results = pd.DataFrame({'Coefficient': coefficients, 'P-Value': p_values})
  results.index = X.columns
  results = results.sort_values(by = 'Coefficient', key = abs, ascending = False) #sort by coeff to determine which coeffs have greatest impact
  results_filtered = results.loc[results['P-Value'] <= alpha] #filter significant coeff by p-value using alpha
  filtered_results_dict[alpha] = results_filtered

In [54]:
for alpha in alpha_values:
  print(f'LS yields {filtered_results_dict[alpha].shape[0]} significant coefficients using alpha = {alpha}')

LS yields 36 significant coefficients using alpha = 0.1
LS yields 28 significant coefficients using alpha = 0.05
LS yields 15 significant coefficients using alpha = 0.01
LS yields 6 significant coefficients using alpha = 0.001


In [55]:
level = 0.05 #take alpha = 5%
significant_coefficients_LS = list(filtered_results_dict[level].index)
filtered_results_dict[level]
#note that the p-value of 0.005 for constants implies that we the intercept is statistically significant (Beta0 = 0.566 != 0).

,Coefficient,P-Value
PersPerOccupHous,0.637574,0.010826
constants,0.566083,0.005393
MedRent,0.370840,0.004287
whitePerCap,-0.354210,0.020414
PctKids2Par,-0.325910,0.036017
PersPerRentOccHous,-0.257846,0.001465
PctEmploy,0.248278,0.001697
RentLowQ,-0.237365,0.000415
MalePctNevMarr,0.230541,0.000712
PctPersDenseHous,0.217437,0.004113


In [56]:
X_filtered = X[significant_coefficients_LS]
X_filtered

,PersPerOccupHous,constants,MedRent,whitePerCap,PctKids2Par,PersPerRentOccHous,PctEmploy,RentLowQ,MalePctNevMarr,PctPersDenseHous,...,NumInShelters,PctIlleg,pctWRetire,MedOwnCostPctIncNoMtg,PctVacMore6Mos,PctEmplManu,PctVacantBoarded,pctWFarmSelf,pctUrban,OtherPerCap
0,0.33,1,0.34,0.39,0.59,0.28,0.68,0.36,0.40,0.09,...,0.04,0.14,0.43,0.25,0.26,0.23,0.05,0.34,1.0,0.36
1,0.17,1,0.37,0.38,0.47,0.17,0.73,0.42,0.63,0.20,...,0.00,0.24,0.39,0.18,0.25,0.57,0.02,0.11,1.0,0.22
2,0.46,1,0.31,0.29,0.44,0.43,0.58,0.27,0.41,0.15,...,0.00,0.46,0.84,0.28,0.30,0.32,0.29,0.19,0.0,0.28
3,0.85,1,0.89,0.40,0.54,1.00,0.71,0.75,0.45,0.12,...,0.00,0.33,0.82,0.47,0.47,0.36,0.60,0.21,1.0,0.36
4,0.59,1,0.38,0.41,0.91,0.37,0.65,0.40,0.27,0.02,...,0.00,0.06,0.71,0.21,0.55,0.67,0.04,0.16,0.9,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.41,1,0.41,0.50,0.68,0.40,0.71,0.45,0.41,0.10,...,0.00,0.11,0.46,0.39,0.21,0.17,0.00,0.14,1.0,0.36
1990,0.74,1,0.54,0.20,0.63,0.87,0.77,0.54,0.75,0.67,...,0.01,0.42,0.48,0.31,0.13,0.10,0.17,0.10,1.0,0.23
1991,0.39,1,0.37,0.34,0.42,0.37,0.46,0.28,0.53,0.15,...,0.06,0.54,0.57,0.53,0.45,0.63,0.53,0.17,1.0,0.22
1992,0.36,1,0.64,0.40,0.65,0.26,0.57,0.58,0.87,0.13,...,0.04,0.17,0.45,0.40,0.34,0.44,0.17,0.21,1.0,0.27


# Best Subsets

## Filtering

In [190]:
print(f'There are {X.shape[1]} explanatory variables in X.  \nTherefore, running Best Subsets is not feasible as this is an NP-hard problem requiring searching through all possible combinations.')

There are 94 explanatory variables in X.  
Therefore, running Best Subsets is not feasible as this is an NP-hard problem requiring searching through all possible combinations.


To filter variables using p-values before performing best subsets selection, start by fitting a least squares (LS) model for each individual variable against the target. From each model, extract the p-value associated with the variable's coefficient. Set a significance threshold, typically 0.05, and exclude variables with p-values above this threshold, as they are less likely to have a meaningful relationship with the target. This process reduces the number of variables, making the subsequent best subsets selection more computationally efficient. Only the significant variables are retained for further modeling.

In [60]:
y = data.loc[:,'ViolentCrimesPerPop']
X = data.loc[:, data.columns != 'ViolentCrimesPerPop']

In [78]:
const = pd.DataFrame([1]*X.shape[0]).rename(columns = {0: 'constants'})

dataframes_Xi = []
for i in range(X.shape[1]):
  dataframes_Xi.append(pd.concat([const, X.iloc[:,i]], axis = 1))

In [149]:
pvals_dict = {'Variable': [], 'Coefficient': [], 'P-Value': []}

In [150]:
for index, var in enumerate(X.columns):
  lsmodel = OLS(y,dataframes_Xi[index]).fit()
  coefficients = lsmodel.params
  p_values = lsmodel.pvalues
  pvals_dict['Variable'].append(var)
  pvals_dict['P-Value'].append(p_values[1:].values[0])
  pvals_dict['Coefficient'].append(coefficients[1:].values[0])

In [154]:
df_bestsubs = pd.DataFrame(pvals_dict)
df_bestsubs

,Variable,Coefficient,P-Value
0,population,0.674060,1.126618e-64
1,householdsize,-0.049698,1.190108e-01
2,racepctblack,0.580310,3.757220e-222
3,racePctWhite,-0.653751,4.835180e-276
4,racePctAsian,0.041964,9.305212e-02
...,...,...,...
95,LandArea,0.418891,7.360161e-19
96,PopDens,0.322807,1.313566e-37
97,PctUsePubTrans,0.156468,4.987116e-12
98,LemasPctOfficDrugUn,0.337975,4.518010e-58


In [171]:
df_bestsubs.loc[df_bestsubs['P-Value'] <= 0.05]

,Variable,Coefficient,P-Value
0,population,0.674060,1.126618e-64
2,racepctblack,0.580310,3.757220e-222
3,racePctWhite,-0.653751,4.835180e-276
5,racePctHisp,0.293672,8.769028e-41
6,agePct12t21,0.090790,6.905984e-03
...,...,...,...
95,LandArea,0.418891,7.360161e-19
96,PopDens,0.322807,1.313566e-37
97,PctUsePubTrans,0.156468,4.987116e-12
98,LemasPctOfficDrugUn,0.337975,4.518010e-58


This method yielded poor results.  We have only disregarded 6 variables by filtering on the p-values.  Now we must try another filtering method.

In [174]:
updating_filtered_vars = df_bestsubs.loc[df_bestsubs['P-Value'] <= 0.05]['Variable'].values
updating_filtered_vars

array(['population', 'racepctblack', 'racePctWhite', 'racePctHisp',
       'agePct12t21', 'agePct12t29', 'agePct16t24', 'agePct65up',
       'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf',
       'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire',
       'medFamInc', 'perCapInc', 'whitePerCap', 'blackPerCap',
       'indianPerCap', 'AsianPerCap', 'HispPerCap', 'NumUnderPov',
       'PctPopUnderPov', 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore',
       'PctUnemployed', 'PctEmploy', 'PctEmplManu', 'PctEmplProfServ',
       'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce',
       'MalePctNevMarr', 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam',
       'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par',
       'PctWorkMom', 'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent',
       'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig',
       'PctRecImmig5', 'PctRecImmig8', 'PctRecImmig10',
       'PctSpeakEnglOnly', 'PctNotSpeakEnglWel

Now we will try eliminating variables based on multicollinearity.  This is done by checking for highly correlated variables. Using techniques like VIF (Variance Inflation Factor) can help to identify variables that are highly correlated and may cause multicollinearity by quanitifying how much the variance of a regression coefficient is inflated due to collinearity with other predictors.  

*   VIF for Each Variable: It calculates how much the variance of the estimated regression coefficients is increased due to multicollinearity.
*   High VIF: If a variable has a high VIF (typically greater than 5 to 10), it indicates that the variable is highly correlated with other variables in the model and may need to be removed.


In [186]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [238]:
y = data.loc[:,'ViolentCrimesPerPop']
X = data.loc[:, updating_filtered_vars] #using already filtered vars from above

# Add a constant to X
X = pd.concat([const, X], axis = 1)

# Compute VIF for each variable
vif_dict = {'Variable': X.columns, 'VIF':[]}

for i in range(X.shape[1]):
  vif_dict['VIF'].append(variance_inflation_factor(X.values, i))

vif_df = pd.DataFrame(vif_dict)
vif_df

,Variable,VIF
0,constants,4333.208550
1,population,289.783005
2,racepctblack,17.245356
3,racePctWhite,20.000628
4,racePctHisp,14.856303
...,...,...
90,LandArea,3.291710
91,PopDens,4.292050
92,PctUsePubTrans,3.197351
93,LemasPctOfficDrugUn,1.575041


In [335]:
vif_level = 5 #The literature suggests that greater then 5 to 10 is high
var_filtered_vif = (vif_df.loc[vif_df['VIF'] <= vif_level].reset_index(drop=True))['Variable'].values
print(f'We have now filtered the variables to the {len(var_filtered_vif)} variables printed below.  However, this is still to many variables to perform best subsets.  We now will try another method to further filter the variables.\n')
var_filtered_vif

We have now filtered the variables to the 24 variables printed below.  However, this is still to many variables to perform best subsets.  We now will try another method to further filter the variables.



array(['pctWFarmSelf', 'pctWRetire', 'blackPerCap', 'indianPerCap',
       'AsianPerCap', 'HispPerCap', 'PctEmplManu', 'PctWorkMom',
       'MedNumBR', 'PctHousOccup', 'PctVacantBoarded', 'MedYrHousBuilt',
       'PctWOFullPlumb', 'MedRentPctHousInc', 'MedOwnCostPctInc',
       'MedOwnCostPctIncNoMtg', 'NumInShelters', 'NumStreet',
       'PctBornSameState', 'LandArea', 'PopDens', 'PctUsePubTrans',
       'LemasPctOfficDrugUn', 'OtherPerCap'], dtype=object)

After filtering features based on p-values and addressing multicollinearity using VIF, it's tempting to assume that collinearity issues have been fully handled. However, VIF examines multicollinearity in the context of the entire set of predictors, rather than directly addressing pairwise correlations. While VIF effectively highlights features that are highly correlated with others in the model, it doesn't explicitly identify pairs of features that might still exhibit strong correlations (e.g., above 0.9).

Therefore, we perform pairwise correlation checks as an additional safeguard to ensure no highly correlated feature pairs remain, which could still lead to issues like multicollinearity, overfitting, or redundancy. This step acts as a final filter to catch any remaining correlations that might not have been flagged by VIF.

In [342]:
X = data.loc[:, var_filtered_vif] #using already filtered vars from above

# Calculate the correlation matrix
correlation_matrix = X.corr()

# Create an upper triangle matrix to avoid duplicate comparisons
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
upper_triangle

,pctWFarmSelf,pctWRetire,blackPerCap,indianPerCap,AsianPerCap,HispPerCap,PctEmplManu,PctWorkMom,MedNumBR,PctHousOccup,...,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctBornSameState,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
pctWFarmSelf,NaN,-0.279462,-0.115485,-0.045322,-0.046613,-0.116150,-0.147322,0.039192,0.064496,-0.033029,...,-0.133666,-0.155505,-0.100780,-0.085159,0.081541,-0.011249,-0.265259,-0.244916,-0.158810,-0.087386
pctWRetire,NaN,NaN,-0.048700,-0.018890,0.045101,0.056074,-0.024798,-0.000719,0.080288,-0.054752,...,-0.185365,0.033598,-0.072032,-0.067954,0.094718,-0.052649,-0.087992,-0.040569,-0.057113,-0.039413
blackPerCap,NaN,NaN,NaN,0.241819,0.332361,0.500448,-0.011909,-0.057701,0.241735,0.275398,...,0.301961,0.000666,-0.075203,-0.044312,-0.184294,-0.058409,0.074431,0.226336,-0.093043,0.342168
indianPerCap,NaN,NaN,NaN,NaN,0.157375,0.217424,-0.025788,-0.004703,0.101196,0.070558,...,0.165779,0.000334,-0.036772,-0.006731,-0.126488,-0.018051,0.040404,0.091937,-0.007364,0.157049
AsianPerCap,NaN,NaN,NaN,NaN,NaN,0.410829,-0.000694,-0.101867,0.225555,0.115863,...,0.106766,0.027664,-0.076207,-0.041568,-0.058530,-0.041211,-0.025155,0.124029,-0.063112,0.183578
HispPerCap,NaN,NaN,NaN,NaN,NaN,NaN,-0.070132,-0.023505,0.263988,0.205077,...,0.201889,0.022057,-0.092878,-0.073295,-0.152939,-0.041593,-0.013839,0.215107,-0.072959,0.485350
PctEmplManu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084156,0.104091,0.296395,...,-0.167125,0.038553,-0.113092,-0.095397,0.335215,-0.113279,0.005384,-0.182148,-0.080246,-0.047193
PctWorkMom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021509,0.064884,...,-0.132022,-0.014137,-0.091668,-0.091525,0.121801,-0.006045,-0.188812,-0.180489,-0.066944,0.015753
MedNumBR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272591,...,-0.065735,-0.003929,-0.216976,-0.187881,0.184142,-0.055368,-0.311659,-0.109770,-0.227716,0.153133
PctHousOccup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.046228,-0.146507,-0.089556,-0.079093,0.170170,-0.177089,0.140485,0.118077,-0.112974,0.125294


In [343]:
correlation_level = 0.8

# Find features with correlation greater than {correlation_level}
high_correlation = [column for column in upper_triangle.columns if any(upper_triangle[column] >= correlation_level)]
low_correlation_vars = [column for column in upper_triangle.columns if any(upper_triangle[column] < correlation_level)]

# Display the removed features
print(f"There were {len(high_correlation)} features removed due to high correlation.")

There were 0 features removed due to high correlation.


In [347]:
X_filtered = X[low_correlation_vars]
X_filtered

,pctWRetire,blackPerCap,indianPerCap,AsianPerCap,HispPerCap,PctEmplManu,PctWorkMom,MedNumBR,PctHousOccup,PctVacantBoarded,...,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctBornSameState,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,0.43,0.32,0.27,0.27,0.41,0.23,0.76,0.5,0.71,0.05,...,0.46,0.25,0.04,0.00,0.42,0.12,0.26,0.20,0.32,0.36
1,0.39,0.33,0.16,0.30,0.35,0.57,0.53,0.0,0.79,0.02,...,0.32,0.18,0.00,0.00,0.50,0.02,0.12,0.45,0.00,0.22
2,0.84,0.27,0.07,0.29,0.39,0.32,0.67,0.5,0.86,0.29,...,0.39,0.28,0.00,0.00,0.49,0.01,0.21,0.02,0.00,0.28
3,0.82,0.39,0.16,0.25,0.44,0.36,0.86,0.5,0.97,0.60,...,0.51,0.47,0.00,0.00,0.30,0.02,0.39,0.28,0.00,0.36
4,0.71,0.28,0.00,0.74,0.48,0.67,0.60,0.5,0.89,0.04,...,0.51,0.21,0.00,0.00,0.72,0.04,0.09,0.02,0.00,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.46,0.27,0.00,0.16,0.52,0.17,0.73,0.5,0.75,0.00,...,0.48,0.39,0.00,0.00,0.28,0.01,0.28,0.05,0.00,0.36
1990,0.48,0.26,0.18,0.20,0.26,0.10,0.40,0.0,0.80,0.17,...,0.67,0.31,0.01,0.00,0.25,0.02,0.37,0.20,0.00,0.23
1991,0.57,0.25,0.19,0.28,0.28,0.63,0.46,0.0,0.65,0.53,...,0.58,0.53,0.06,0.02,0.68,0.08,0.32,0.18,0.91,0.22
1992,0.45,0.29,0.20,0.31,0.37,0.44,0.63,0.0,0.84,0.17,...,0.44,0.40,0.04,0.01,0.64,0.03,0.38,0.33,0.22,0.27


## Running Best Subsets

Now to run best subsets on the variables that we have filtered.

In [348]:
import time
start_time = time.time()

colnames = X_filtered.columns
# Define a linear regression model
linear_reg = LinearRegression()
#Obtain exhaustive mse values
efs = EFS(
      linear_reg,
      min_features=1,
      max_features=8,
      scoring="neg_mean_squared_error", #we use negative mse since the sfs method maximizes the score
      cv=0
)
efs.fit(X_filtered, y)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time:.4f} seconds")

KeyboardInterrupt: 

In [ ]:
#Obtain a summary of the exhaustive results (all possible models)
efs_summary = pd.DataFrame.from_dict(efs.get_metric_dict()).T[["feature_idx", "avg_score"]]
efs_summary["feature_idx"]
efs_size = np.zeros(len(efs_summary))
for i in range(len(efs_summary)):
    efs_size[i] = len(efs_summary["feature_idx"][i])
efs_summary['model size'] = efs_size
efs_summary.rename(columns={'avg_score': 'neg_mse'}, inplace=True)
print("The first few results for exhaustive feature selection (all possible models) look like:")
print(efs_summary.iloc[range(15)].to_string(index=False))

In [ ]:
#code to calculate model evaluation criteria for all models in efs_summary

#code to calculate sigma hat squared for Mallows Cp calculation
X_with_const = sm.tools.tools.add_constant(X)
model = sm.regression.linear_model.OLS(y, X_with_const).fit()
sigma_hat_squared = np.sum(model.resid ** 2) / (len(y.to_numpy()) - len(X.to_numpy()[0])) #intercept not counted as predictor
#code to calculate total sum of squares, used for r2 calculation:
Y_mean = np.mean(y)
tss = sum((y - np.full(len(y), Y_mean))**2)

prederror = np.zeros((len(efs_summary),4))
for i in range(len(efs_summary)):
    rss = -efs_summary.iloc[i,1]*len(X)
    p = efs_summary.iloc[i,2]
    n = len(X)
    aic = 2*p + n * np.log(rss/n)
    bic = p * np.log(n) + n * np.log(rss/n)
    mallows_cp = (rss / sigma_hat_squared) - n + 2 * p
    r2 = 1 - rss/tss
    adjr2 = 1 - (1 - r2)*(n-1)/(n-p-1)
    prederror[i,0] = bic
    prederror[i,1] = adjr2
    prederror[i,2] = mallows_cp
    prederror[i,3] = aic

efs_summary['bic'] = prederror[:,0]
efs_summary['adjr2'] = prederror[:,1]
efs_summary['cp'] = prederror[:,2]
efs_summary['aic'] = prederror[:,3]
print(efs_summary.iloc[0:15,:].to_string(index=False))

Our best model has the following:


*   Maximum neg_mse
*   low BIC & AIC (we will just focus on BIC)
*   High adjr2

# Ridge

Standardize Data.

In [ ]:
# Center y
Y_centered = y - np.mean(y)

# Center X (not scaled)
X_centered = X - np.mean(X, axis=0)

# Standardize X (centered and scaled)
scaler = StandardScaler(with_mean=True, with_std=True)
X_standardized = scaler.fit_transform(X)

In [ ]:
lam = 10 #hyperparameter tuning
ridge = Ridge(alpha = lam, fit_intercept = False)
ridge.fit(X_standardized, Y_centered)
beta = ridge.coef_

lambdas = np.exp(np.linspace(np.log(0.01), np.log(100), 100))
betasr = np.zeros((len(lambdas), X_final.shape[1]))
for i, lamb in enumerate(lambdas):
    betasr[i, :] = np.linalg.inv(X_standardized.T @ X_standardized + np.diag([n*lamb] * X_standardized.shape[1])) @ (X_standardized.T @ Y_centered.to_numpy() / n).ravel()

# Plot Ridge paths
plt.figure(figsize=(10, 6))
for j in range(X_standardized.shape[1]):
    plt.plot(lambdas, betasr[:, j], label=f"Variable {j+1}")
plt.xscale("log")
plt.xlabel("Log Lambda")
plt.ylabel("Coefficients")
plt.title("Ridge Regression Coefficient Paths")
plt.show()


# LASSO.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso

# Fit OLS
fit0 = LinearRegression()
fit0.fit(X_standardized, Y_centered)

# Fit LASSO
lasso = Lasso(fit_intercept=False)
lasso.fit(X_standardized, Y_centered)

# Ensure all arrays have the same shape
colnames = np.array(X.columns)  # Convert to NumPy array
ols_coefs = fit0.coef_.flatten()  # Ensure it's 1D
lasso_coefs = lasso.coef_.flatten()  # Ensure it's 1D

# Stack into a DataFrame
df = pd.DataFrame({
    "Variables": colnames,
    "Without L1": ols_coefs,
    "With L1": lasso_coefs
})

In [ ]:
lasso = Lasso(alpha=0.0001, fit_intercept=False)
lasso.fit(X_standardized, Y_centered)

In [ ]:
lasso = Lasso(alpha=0.01, fit_intercept=True)  # Keep intercept
lasso.fit(X_final, y)  # Use unstandardized features

lasso.coef_[lasso.coef_ != 0]

beta = pd.DataFrame(lasso.coef_)
non_zero_coef_LASSO = []
index_nonzero_coef = []
for index, value in enumerate(beta[0]):
  if value != 0:
    non_zero_coef_LASSO.append((index, value))
    index_nonzero_coef.append(index)

sig_vars_LASSO = list(X_final.columns[index_nonzero_coef])

print(f'The significant (nonzero) coefficients given by LASSO are {sig_vars_LASSO} at lambda = 0.01')